<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/chuongmep/aps-toolkit/blob/dev/APSToolkitPython/Tutorials/04.01.%20Explore%20Revit%20Data%20With%20APSToolkit.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/chuongmep/aps-toolkit/blob/dev/APSToolkitPython/Tutorials/04.01.%20Explore%20Revit%20Data%20With%20APSToolkit.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<table align="left">
  <tr>
    <td>Author(s)</td>
    <td><a href="https://github.com/chuongmep" id="author-link">Chuong Ho</a></td>
    <!-- <td><a href="<your repo>" id="github-link">YourName</a></td> -->
  </tr>
</table>

In [1]:
%pip install aps-toolkit --upgrade

In [2]:
from aps_toolkit import Auth
import pandas as pd

### Authentication

In [3]:
token = Auth().auth2leg()

#### Auth Via Google Colab

In [ ]:
# from google.colab import userdata
# client_id = userdata.get('APS_CLIENT_ID')
# client_secret = userdata.get('APS_CLIENT_SECRET')
# auth = Auth(client_id, client_secret)
# token = auth.auth2leg()

In [4]:
urn = "dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLk9kOHR4RGJLU1NlbFRvVmcxb2MxVkE_dmVyc2lvbj0yNA"

### Reader

#### General Reader with Default View

In [5]:
from aps_toolkit import PropDbReaderRevit

prop_reader = PropDbReaderRevit(urn, token)

#### Reader with Custom View

In [6]:
from aps_toolkit import Derivative
derivative = Derivative(urn, token,"US")

In [7]:
items = derivative.read_svf_manifest_items()
for item in items:
    print(item.path_info.root_file_name)

In [8]:
item = items[0]
print("Guid: ", item.guid)
print("Root File Name Item: ", item.path_info.root_file_name)
print("Path: ", item.path_info.files)

In [9]:
prop_reader = PropDbReaderRevit(urn, token,"US",items[0])

In [20]:
df = prop_reader.get_data_by_categories_and_params(["Doors", "Windows"], ["Width","Category", "Height", "Length", "Workset"], True)
pd.set_option('display.max_columns', 5)
pd.set_option('display.width', 1000)
df

In [21]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [22]:
agent = create_pandas_dataframe_agent(ChatOpenAI(temperature=0), df, verbose=True)

In [23]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [24]:
agent.invoke("how many category there? what kind of categories?")